In [ ]:
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
from quantopian.pipeline.filters import Q1500US

In [ ]:
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
from quantopian.pipeline.filters import Q1500US
from quantopian.pipeline.factors import SimpleMovingAverage
from quantopian.pipeline.data.builtin import USEquityPricing


def make_pipeline():
    
    universe = (Q1500US())
    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=10)
    
    pipe = Pipeline(
        columns = {'10_day_mean_close': mean_close_10},
        screen=universe)
    #pipe = Pipeline()
    
    return pipe

results = run_pipeline(make_pipeline(),start_date='2015-05-05', end_date='2016-05-05' )
len(results)


In [ ]:
results.head()

In [ ]:
results.tail()

In [ ]:
assets = results.index.levels[1].unique()

In [ ]:
pricing = get_pricing(assets,start_date='2015-05-05', end_date='2016-05-05', fields='open_price'  )

In [ ]:
pricing.head()

In [ ]:
import alphalens 
alphalens.tears.create_factor_tear_sheet(factor=results['10_day_mean_close'], 
                                        prices=pricing, 
                                        quantiles=10, 
                                         periods=(1,5,10)
                                        )

In [ ]:
bt = get_backtest('590aff0e1f910765c7040cb0')#59052109ec57de662dfe4f35


In [ ]:
bt.positions[bt.positions.sid==36144][['cost_basis', 'last_sale_price']].plot()


In [ ]:
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
from quantopian.pipeline.filters.morningstar import Q1500US 
from quantopian.pipeline.data.sentdex import sentiment
from quantopian.pipeline.data.morningstar import operation_ratios

def  make_pipeline():
    # Yes: operation_ratios.revenue_growth.latest 
    
    testing_factor1 = operation_ratios.operation_margin.latest
    testing_factor2 = operation_ratios.revenue_growth.latest
    testing_factor3 = sentiment.sentiment_signal.latest
    
    universe = (Q1500US() 
                & testing_factor1.notnull()
                & testing_factor2.notnull()
                & testing_factor3.notnull())
    
    testing_factor1 = testing_factor1.rank(mask=universe, method='average')
    testing_factor2 = testing_factor2.rank(mask=universe, method='average')
    testing_factor3 = testing_factor3.rank(mask=universe, method='average')
    
    testing_factor = testing_factor1 + testing_factor2 + testing_factor3
    
    testing_quantile = testing_factor.quantiles(2)
    
    
    pipe = Pipeline(columns={'testing_factor': testing_factor,
                            'shorts': testing_quantile.eq(0),
                            'longs':  testing_quantile.eq(1),},
                    screen=universe)
    
    return pipe

results = run_pipeline(make_pipeline(), start_date='2015-1-1', end_date='2016-1-1')
results.head()

In [ ]:
assets = results.index.levels[1].unique()
pricing = get_pricing(assets, start_date='2015-1-1', end_date='2016-1-1', fields='open_price')
len(pricing)

In [ ]:
import alphalens

alphalens.tears.create_factor_tear_sheet(factor=results['testing_factor'], 
                                        prices=pricing, 
                                        quantiles=2,
                                        periods=(3,10,30))